### Teste do servidor FastAPI

In [ ]:
# Requirements
!pip install requests

In [ ]:
# Script para testar se o servidor RAG está funcionando

import requests
import time
import sys

def test_server():
    base_url = "http://localhost:8001"  # Ajuste a porta conforme necessário
    
    print(f"🧪 Testando servidor em: {base_url}")
    print("=" * 50)
    
    # Teste 1: Rota raiz
    try:
        print("1️⃣ Testando rota raiz (/)...")
        response = requests.get(f"{base_url}/", timeout=5)
        print(f"   Status: {response.status_code}")
        print(f"   Resposta: {response.text[:200]}...")
        print()
    except requests.exceptions.ConnectionError:
        print("   ❌ Erro: Não foi possível conectar ao servidor")
        print("   💡 Verifique se o servidor está rodando na porta correta")
        return False
    except Exception as e:
        print(f"   ❌ Erro: {e}")
        return False
    
    # Teste 2: Health check
    try:
        print("2️⃣ Testando health check (/health)...")
        response = requests.get(f"{base_url}/health", timeout=5)
        print(f"   Status: {response.status_code}")
        print(f"   Resposta: {response.text}")
        print()
    except Exception as e:
        print(f"   ❌ Erro: {e}")
    
    # Teste 3: Documentação
    try:
        print("3️⃣ Testando documentação (/docs)...")
        response = requests.get(f"{base_url}/docs", timeout=5)
        print(f"   Status: {response.status_code}")
        if response.status_code == 200:
            print("   ✅ Documentação acessível")
        else:
            print("   ⚠️  Documentação não encontrada")
        print()
    except Exception as e:
        print(f"   ❌ Erro: {e}")
    
    # Teste 4: OpenAPI JSON
    try:
        print("4️⃣ Testando OpenAPI JSON (/openapi.json)...")
        response = requests.get(f"{base_url}/openapi.json", timeout=5)
        print(f"   Status: {response.status_code}")
        if response.status_code == 200:
            print("   ✅ OpenAPI JSON acessível")
        else:
            print("   ⚠️  OpenAPI JSON não encontrado")
        print()
    except Exception as e:
        print(f"   ❌ Erro: {e}")
    
    print("=" * 50)
    print("✅ Testes concluídos!")
    print(f"📚 Acesse a documentação em: {base_url}/docs")
    
    return True

print("⏳ Aguardando servidor inicializar...")
time.sleep(2)

success = test_server()

if not success:
    print("\n🔧 Soluções possíveis:")
    print("1. Verifique se o servidor está rodando")
    print("2. Verifique se a porta está correta")
    print("3. Verifique se não há conflito de portas")
    print("4. Verifique os logs do servidor")
    sys.exit(1) 

In [2]:
# Enviar mensagem para o servidor

message = "Olá, meu nome é Marcelo, que horas são?"

import requests
import datetime

url = "http://127.0.0.1:8001/message"

payload = {
    "phone_number": "5582999464789",
    "message": message,
    "message_id": "abc123",
    "user_name": "Teste",
    "timestamp": datetime.datetime.now().isoformat()
}

response = requests.post(url, json=payload)

print("Status code:", response.status_code)
print("Resposta:", response.json())

Status code: 200
Resposta: {'status': 'success', 'message': 'Mensagem recebida e será processada em background'}


In [ ]:
# Enviar mensagem para base de conhecimento

message = "Olá, meu nome é Marcelo, que horas são?"

import requests
import datetime

url = "http://127.0.0.1:8000/add-knowledge"

payload = {
    "phone_number": "5582999464789",
    "message": message,
    "message_id": "abc123",
    "user_name": "Teste",
    "timestamp": datetime.datetime.now().isoformat()
}

response = requests.post(url, json=payload)

print("Status code:", response.status_code)
print("Resposta:", response.json())

### SUPABASE

In [ ]:
# Teste do Supabase
from supabase import create_client
import os
from dotenv import load_dotenv

load_dotenv(".env")

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_ANON_KEY")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

data = {
    "phone_number": "11999999999",
    "user_name": "Teste",
    "status": "active"
}
result = supabase.table("conversations").insert(data).execute()
print(result)

In [ ]:
# SELECT * FROM conversations LIMIT 1
result = supabase.table("conversations").select("*").limit(1).execute()
print(result.data)

### OLLAMA

In [ ]:
# Teste o Modelo
import os
from dotenv import load_dotenv
import requests

load_dotenv(".env")

OLLAMA_URL = os.getenv("OLLAMA_URL")

response = requests.post(
    f"{OLLAMA_URL}/api/generate",
    json={
        "model": "llama3.2",
        "prompt": "Olá, quem é você?"
    }
)
print(response)

In [ ]:
# CUDA

import torch
print(torch.cuda.is_available())  # Deve retornar True
#print(torch.cuda.current_device())  # Deve retornar 0 (ou outro índice, se múltiplas GPUs)
print(torch.cuda.get_device_name(0))  # Nome da sua GPU

### PUB/SUB

In [14]:
# Publicar mensagem diretamente no PubSub

import os
from google.cloud import pubsub_v1

# Configurar credenciais e variáveis
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Projetos\python-llama-rag-wts\credenciais.json"
PROJECT_ID = os.getenv("GCP_PROJECT_ID", "treinamentoia-465816")
PUBSUB_TOPIC_ID = os.getenv("PUBSUB_TOPIC_ID", "whatsapp-messages")
SUBSCRIPTION_ID = os.getenv("PUBSUB_SUBSCRIPTION_ID", "whatsapp-messages-sub")

In [ ]:
# Publicar mensagem diretamente no PubSub
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(PROJECT_ID, PUBSUB_TOPIC_ID)

data = "Mensagem de teste do PubSub".encode("utf-8")
future = publisher.publish(topic_path, data)
print("Mensagem publicada! ID:", future.result())

In [ ]:
# Confirmar todas as mensagens pendentes
subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(PROJECT_ID, SUBSCRIPTION_ID)

try:
    while True:
        # Tentar receber mensagens
        request = {
            "subscription": subscription_path,
            "max_messages": 10,  # Pega até 10 mensagens por vez
            "return_immediately": True
        }
        
        response = subscriber.pull(request=request, timeout=5.0)
        print(f'{len(response.received_messages)} mensages lidas')
        
        if not response.received_messages:
            print("Nenhuma mensagem pendente encontrada.")
            break
        
        # Fazer ack de todas as mensagens recebidas
        ack_ids = [msg.ack_id for msg in response.received_messages]
        

        subscriber.acknowledge(request={"subscription": subscription_path, "ack_ids": ack_ids})
        
    

except Exception as e:
    print(f"Erro ao limpar fila: {e}")